<a href="https://colab.research.google.com/github/arunabha-mandal-1/bnlp-basics/blob/main/Lemmatization1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

There are few Bengali Lemmatizer in BNLP domain. Let's try!

In [1]:
text1 = """জীবের মধ্যে সবচেয়ে সম্পূর্ণতা মানুষের। কিন্তু সবচেয়ে অসম্পূর্ণ হয়ে সে জন্মগ্রহণ করে। বাঘ ভালুক তার জীবনযাত্রার পনেরো- আনা মূলধন নিয়ে আসে প্রকৃতির মালখানা থেকে। জীবরঙ্গভূমিতে মানুষ এসে দেখা দেয় দুই শূন্য হাতে মুঠো বেঁধে।
   মানুষ আসবার পূর্বেই জীবসৃষ্টিযজ্ঞে প্রকৃতির ভূরিব্যয়ের পালা শেষ হয়ে এসেছে। বিপুল মাংস, কঠিন বর্ম, প্রকাণ্ড লেজ নিয়ে জলে স্থলে পৃথুল দেহের যে অমিতাচার প্রবল হয়ে উঠেছিল তাতে ধরিত্রীকে দিলে ক্লান্ত করে। প্রমাণ হল আতিশয্যের পরাভব অনিবার্য। পরীক্ষায় এটাও স্থির হয়ে গেল যে, প্রশ্রয়ের পরিমাণ যত বেশি হয় দুর্বলতার বোঝাও তত দুর্বহ হয়ে ওঠে। নূতন পর্বে প্রকৃতি যথাসম্ভব মানুষের বরাদ্দ কম করে দিয়ে নিজে রইল নেপথ্যে।
   মানুষকে দেখতে হল খুব ছোটো, কিন্তু সেটা একটা কৌশল মাত্র। এবারকার জীবযাত্রার পালায় বিপুলতাকে করা হল বহুলতায় পরিণত। মহাকায় জন্তু ছিল প্রকাণ্ড একলা, মানুষ হল দূরপ্রসারিত অনেক।"""

In [2]:
text2 = """চিত্ত যেথা ভয়শূন্য, উচ্চ যেথা শির,
জ্ঞান যেথা মুক্ত, যেথা গৃহের প্রাচীর আপন প্রাঙ্গণতলে দিবসশর্বরী বসুধারে রাখে নাই খণ্ড ক্ষুদ্র করি,
যেথা বাক্য হৃদয়ের উৎসমুখ হতে উচ্ছ্বসিয়া উঠে,
যেথা নির্বারিত স্রোতে দেশের হ্রদয়
বুদ্ধির সূত্রে শত শাখা শত স্রোতে সেঁচিয়া উঠে,
যেথা কর্মযজ্ঞে বসন্তের হাওয়ায় অনুপ্রাণিত হয়ে অজানা পথে চলবারে জনে,
যেথা চিন্তা তোমার আচারপন্থাকে বুদ্ধির তীক্ষ্ণ তলওয়ারে প্রতি দিন আঘাত করে,
যেথা তুমি বিশ্ববিধাতা প্রবুদ্ধ উদার সৃষ্টি করে থাকো
সর্বত্র সেই স্বর্গে উঠিতে যাক আমার দেশ।"""

# Spark NLP by John Snow LABS

In [3]:
! pip install -q pyspark==3.3.0 spark-nlp==4.2.8

In [4]:
import json
import pandas as pd
import numpy as np

import sparknlp
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

In [5]:
spark = sparknlp.start()

In [6]:
sparknlp.version()

'4.2.8'

In [7]:
spark

In [8]:
#getting lemma files
#don't need this for bengali
!wget https://raw.githubusercontent.com/mahavivo/vocabulary/master/lemmas/AntBNC_lemmas_ver_001.txt

--2024-05-23 15:36:11--  https://raw.githubusercontent.com/mahavivo/vocabulary/master/lemmas/AntBNC_lemmas_ver_001.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1348552 (1.3M) [text/plain]
Saving to: ‘AntBNC_lemmas_ver_001.txt.1’

AntBNC_lemmas_ver_0 100%[===================>]   1.29M  --.-KB/s    in 0.07s   

2024-05-23 15:36:11 (19.3 MB/s) - ‘AntBNC_lemmas_ver_001.txt.1’ saved [1348552/1348552]



In [9]:
# Defining Spark NLP pipeline
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# sentenceDetector = SentenceDetector()\
#     .setInputCols(['document'])\
#     .setOutputCol('sentences')

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

# normalizer = Normalizer() \
#     .setInputCols(["token"]) \
#     .setOutputCol("normalized")\
#     .setLowercase(True)\
#     .setCleanupPatterns(["[^\w\d\s]"])

# stopwords_cleaner = StopWordsCleaner()\
#     .setInputCols("token")\
#     .setOutputCol("removed_stopwords")\
#     .setCaseSensitive(False)\

# stemmer = Stemmer() \
#     .setInputCols(["token"]) \
#     .setOutputCol("stem")

lemmatizer = LemmatizerModel.pretrained("lemma", "bn") \
    .setInputCols(["token"]) \
    .setOutputCol("lemma") \
    # .setDictionary("./AntBNC_lemmas_ver_001.txt", value_delimiter ="\t", key_delimiter = "->")

nlp_pipeline = Pipeline(stages=[documentAssembler,
                              #  sentenceDetector,
                               tokenizer,
                              #  normalizer,
                              #  stopwords_cleaner,
                              #  stemmer,
                               lemmatizer])

light_pipeline = LightPipeline(nlp_pipeline.fit(spark.createDataFrame([[""]]).toDF("text")))


lemma download started this may take some time.
Approximate size to download 90.6 KB
[OK!]


In [10]:
# Run pipeline
result1 = light_pipeline.fullAnnotate([text1])
result2 = light_pipeline.fullAnnotate([text2])

Before _validateStagesInputCols
Before _validateStagesInputCols


In [11]:
result1

[{'document': [Annotation(document, 0, 818, জীবের মধ্যে সবচেয়ে সম্পূর্ণতা মানুষের। কিন্তু সবচেয়ে অসম্পূর্ণ হয়ে সে জন্মগ্রহণ করে। বাঘ ভালুক তার জীবনযাত্রার পনেরো- আনা মূলধন নিয়ে আসে প্রকৃতির মালখানা থেকে। জীবরঙ্গভূমিতে মানুষ এসে দেখা দেয় দুই শূন্য হাতে মুঠো বেঁধে।
      মানুষ আসবার পূর্বেই জীবসৃষ্টিযজ্ঞে প্রকৃতির ভূরিব্যয়ের পালা শেষ হয়ে এসেছে। বিপুল মাংস, কঠিন বর্ম, প্রকাণ্ড লেজ নিয়ে জলে স্থলে পৃথুল দেহের যে অমিতাচার প্রবল হয়ে উঠেছিল তাতে ধরিত্রীকে দিলে ক্লান্ত করে। প্রমাণ হল আতিশয্যের পরাভব অনিবার্য। পরীক্ষায় এটাও স্থির হয়ে গেল যে, প্রশ্রয়ের পরিমাণ যত বেশি হয় দুর্বলতার বোঝাও তত দুর্বহ হয়ে ওঠে। নূতন পর্বে প্রকৃতি যথাসম্ভব মানুষের বরাদ্দ কম করে দিয়ে নিজে রইল নেপথ্যে।
      মানুষকে দেখতে হল খুব ছোটো, কিন্তু সেটা একটা কৌশল মাত্র। এবারকার জীবযাত্রার পালায় বিপুলতাকে করা হল বহুলতায় পরিণত। মহাকায় জন্তু ছিল প্রকাণ্ড একলা, মানুষ হল দূরপ্রসারিত অনেক।, {}, [])],
  'token': [Annotation(token, 0, 4, জীবের, {'sentence': '0'}, []),
   Annotation(token, 6, 10, মধ্যে, {'sentence': '0'}

In [12]:
result2

[{'document': [Annotation(document, 0, 499, চিত্ত যেথা ভয়শূন্য, উচ্চ যেথা শির,
   জ্ঞান যেথা মুক্ত, যেথা গৃহের প্রাচীর আপন প্রাঙ্গণতলে দিবসশর্বরী বসুধারে রাখে নাই খণ্ড ক্ষুদ্র করি,
   যেথা বাক্য হৃদয়ের উৎসমুখ হতে উচ্ছ্বসিয়া উঠে,
   যেথা নির্বারিত স্রোতে দেশের হ্রদয়
   বুদ্ধির সূত্রে শত শাখা শত স্রোতে সেঁচিয়া উঠে,
   যেথা কর্মযজ্ঞে বসন্তের হাওয়ায় অনুপ্রাণিত হয়ে অজানা পথে চলবারে জনে,
   যেথা চিন্তা তোমার আচারপন্থাকে বুদ্ধির তীক্ষ্ণ তলওয়ারে প্রতি দিন আঘাত করে,
   যেথা তুমি বিশ্ববিধাতা প্রবুদ্ধ উদার সৃষ্টি করে থাকো
   সর্বত্র সেই স্বর্গে উঠিতে যাক আমার দেশ।, {}, [])],
  'token': [Annotation(token, 0, 4, চিত্ত, {'sentence': '0'}, []),
   Annotation(token, 6, 9, যেথা, {'sentence': '0'}, []),
   Annotation(token, 11, 18, ভয়শূন্য, {'sentence': '0'}, []),
   Annotation(token, 19, 19, ,, {'sentence': '0'}, []),
   Annotation(token, 21, 24, উচ্চ, {'sentence': '0'}, []),
   Annotation(token, 26, 29, যেথা, {'sentence': '0'}, []),
   Annotation(token, 31, 33, শির, {'sentence': '0'}, []),
 

In [13]:
type(result2)

list

In [14]:
len(result2)

1

In [15]:
word_tokens1 = [a.result for a in result1[0]['token']]
print(word_tokens1)

['জীবের', 'মধ্যে', 'সবচেয়ে', 'সম্পূর্ণতা', 'মানুষের।', 'কিন্তু', 'সবচেয়ে', 'অসম্পূর্ণ', 'হয়ে', 'সে', 'জন্মগ্রহণ', 'করে।', 'বাঘ', 'ভালুক', 'তার', 'জীবনযাত্রার', 'পনেরো', '-', 'আনা', 'মূলধন', 'নিয়ে', 'আসে', 'প্রকৃতির', 'মালখানা', 'থেকে।', 'জীবরঙ্গভূমিতে', 'মানুষ', 'এসে', 'দেখা', 'দেয়', 'দুই', 'শূন্য', 'হাতে', 'মুঠো', 'বেঁধে।', 'মানুষ', 'আসবার', 'পূর্বেই', 'জীবসৃষ্টিযজ্ঞে', 'প্রকৃতির', 'ভূরিব্যয়ের', 'পালা', 'শেষ', 'হয়ে', 'এসেছে।', 'বিপুল', 'মাংস', ',', 'কঠিন', 'বর্ম', ',', 'প্রকাণ্ড', 'লেজ', 'নিয়ে', 'জলে', 'স্থলে', 'পৃথুল', 'দেহের', 'যে', 'অমিতাচার', 'প্রবল', 'হয়ে', 'উঠেছিল', 'তাতে', 'ধরিত্রীকে', 'দিলে', 'ক্লান্ত', 'করে।', 'প্রমাণ', 'হল', 'আতিশয্যের', 'পরাভব', 'অনিবার্য।', 'পরীক্ষায়', 'এটাও', 'স্থির', 'হয়ে', 'গেল', 'যে', ',', 'প্রশ্রয়ের', 'পরিমাণ', 'যত', 'বেশি', 'হয়', 'দুর্বলতার', 'বোঝাও', 'তত', 'দুর্বহ', 'হয়ে', 'ওঠে।', 'নূতন', 'পর্বে', 'প্রকৃতি', 'যথাসম্ভব', 'মানুষের', 'বরাদ্দ', 'কম', 'করে', 'দিয়ে', 'নিজে', 'রইল', 'নেপথ্যে।', 'মানুষকে', 'দেখতে', 'হল', 'খুব', 'ছোটো', ',', '

Tokenize using Spark NLP

In [16]:
data1 = spark.createDataFrame([[text1]]).toDF("text")
documentAssembler1 = DocumentAssembler().setInputCol("text").setOutputCol("document")
tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token").fit(data1)
pipeline = Pipeline().setStages([documentAssembler, tokenizer]).fit(data1)
result = pipeline.transform(data1)
result.selectExpr("token.result").show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                      

In [17]:
type(result)

pyspark.sql.dataframe.DataFrame

In [18]:
result.select("token.result") # Trying to get it as an list

DataFrame[result: array<string>]

# Another Approach

In [19]:
!pip install bnlp_toolkit

In [20]:
from bnlp import NLTKTokenizer
bnltk = NLTKTokenizer()

punkt not found. downloading...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [21]:
bnlp_word_tokens1 = bnltk.word_tokenize(text1)
bnlp_word_tokens2 = bnltk.word_tokenize(text2)

In [22]:
bnlp_word_tokens1

['জীবের',
 'মধ্যে',
 'সবচেয়ে',
 'সম্পূর্ণতা',
 'মানুষের',
 '।',
 'কিন্তু',
 'সবচেয়ে',
 'অসম্পূর্ণ',
 'হয়ে',
 'সে',
 'জন্মগ্রহণ',
 'করে',
 '।',
 'বাঘ',
 'ভালুক',
 'তার',
 'জীবনযাত্রার',
 'পনেরো-',
 'আনা',
 'মূলধন',
 'নিয়ে',
 'আসে',
 'প্রকৃতির',
 'মালখানা',
 'থেকে',
 '।',
 'জীবরঙ্গভূমিতে',
 'মানুষ',
 'এসে',
 'দেখা',
 'দেয়',
 'দুই',
 'শূন্য',
 'হাতে',
 'মুঠো',
 'বেঁধে',
 '।',
 'মানুষ',
 'আসবার',
 'পূর্বেই',
 'জীবসৃষ্টিযজ্ঞে',
 'প্রকৃতির',
 'ভূরিব্যয়ের',
 'পালা',
 'শেষ',
 'হয়ে',
 'এসেছে',
 '।',
 'বিপুল',
 'মাংস',
 ',',
 'কঠিন',
 'বর্ম',
 ',',
 'প্রকাণ্ড',
 'লেজ',
 'নিয়ে',
 'জলে',
 'স্থলে',
 'পৃথুল',
 'দেহের',
 'যে',
 'অমিতাচার',
 'প্রবল',
 'হয়ে',
 'উঠেছিল',
 'তাতে',
 'ধরিত্রীকে',
 'দিলে',
 'ক্লান্ত',
 'করে',
 '।',
 'প্রমাণ',
 'হল',
 'আতিশয্যের',
 'পরাভব',
 'অনিবার্য',
 '।',
 'পরীক্ষায়',
 'এটাও',
 'স্থির',
 'হয়ে',
 'গেল',
 'যে',
 ',',
 'প্রশ্রয়ের',
 'পরিমাণ',
 'যত',
 'বেশি',
 'হয়',
 'দুর্বলতার',
 'বোঝাও',
 'তত',
 'দুর্বহ',
 'হয়ে',
 'ওঠে',
 '।',
 'নূতন',
 'পর্বে',
 'প্রকৃতি',
 

In [23]:
bnlp_word_tokens2

['চিত্ত',
 'যেথা',
 'ভয়শূন্য',
 ',',
 'উচ্চ',
 'যেথা',
 'শির',
 ',',
 'জ্ঞান',
 'যেথা',
 'মুক্ত',
 ',',
 'যেথা',
 'গৃহের',
 'প্রাচীর',
 'আপন',
 'প্রাঙ্গণতলে',
 'দিবসশর্বরী',
 'বসুধারে',
 'রাখে',
 'নাই',
 'খণ্ড',
 'ক্ষুদ্র',
 'করি',
 ',',
 'যেথা',
 'বাক্য',
 'হৃদয়ের',
 'উৎসমুখ',
 'হতে',
 'উচ্ছ্বসিয়া',
 'উঠে',
 ',',
 'যেথা',
 'নির্বারিত',
 'স্রোতে',
 'দেশের',
 'হ্রদয়',
 'বুদ্ধির',
 'সূত্রে',
 'শত',
 'শাখা',
 'শত',
 'স্রোতে',
 'সেঁচিয়া',
 'উঠে',
 ',',
 'যেথা',
 'কর্মযজ্ঞে',
 'বসন্তের',
 'হাওয়ায়',
 'অনুপ্রাণিত',
 'হয়ে',
 'অজানা',
 'পথে',
 'চলবারে',
 'জনে',
 ',',
 'যেথা',
 'চিন্তা',
 'তোমার',
 'আচারপন্থাকে',
 'বুদ্ধির',
 'তীক্ষ্ণ',
 'তলওয়ারে',
 'প্রতি',
 'দিন',
 'আঘাত',
 'করে',
 ',',
 'যেথা',
 'তুমি',
 'বিশ্ববিধাতা',
 'প্রবুদ্ধ',
 'উদার',
 'সৃষ্টি',
 'করে',
 'থাকো',
 'সর্বত্র',
 'সেই',
 'স্বর্গে',
 'উঠিতে',
 'যাক',
 'আমার',
 'দেশ',
 '।']

In [24]:
bnlp_word_tokens1 = [word for word in bnlp_word_tokens1 if (word != "," and word != "।")]
bnlp_word_tokens2 = [word for word in bnlp_word_tokens2 if (word != "," and word != "।")]

In [26]:
bnlp_word_tokens1

['জীবের',
 'মধ্যে',
 'সবচেয়ে',
 'সম্পূর্ণতা',
 'মানুষের',
 'কিন্তু',
 'সবচেয়ে',
 'অসম্পূর্ণ',
 'হয়ে',
 'সে',
 'জন্মগ্রহণ',
 'করে',
 'বাঘ',
 'ভালুক',
 'তার',
 'জীবনযাত্রার',
 'পনেরো-',
 'আনা',
 'মূলধন',
 'নিয়ে',
 'আসে',
 'প্রকৃতির',
 'মালখানা',
 'থেকে',
 'জীবরঙ্গভূমিতে',
 'মানুষ',
 'এসে',
 'দেখা',
 'দেয়',
 'দুই',
 'শূন্য',
 'হাতে',
 'মুঠো',
 'বেঁধে',
 'মানুষ',
 'আসবার',
 'পূর্বেই',
 'জীবসৃষ্টিযজ্ঞে',
 'প্রকৃতির',
 'ভূরিব্যয়ের',
 'পালা',
 'শেষ',
 'হয়ে',
 'এসেছে',
 'বিপুল',
 'মাংস',
 'কঠিন',
 'বর্ম',
 'প্রকাণ্ড',
 'লেজ',
 'নিয়ে',
 'জলে',
 'স্থলে',
 'পৃথুল',
 'দেহের',
 'যে',
 'অমিতাচার',
 'প্রবল',
 'হয়ে',
 'উঠেছিল',
 'তাতে',
 'ধরিত্রীকে',
 'দিলে',
 'ক্লান্ত',
 'করে',
 'প্রমাণ',
 'হল',
 'আতিশয্যের',
 'পরাভব',
 'অনিবার্য',
 'পরীক্ষায়',
 'এটাও',
 'স্থির',
 'হয়ে',
 'গেল',
 'যে',
 'প্রশ্রয়ের',
 'পরিমাণ',
 'যত',
 'বেশি',
 'হয়',
 'দুর্বলতার',
 'বোঝাও',
 'তত',
 'দুর্বহ',
 'হয়ে',
 'ওঠে',
 'নূতন',
 'পর্বে',
 'প্রকৃতি',
 'যথাসম্ভব',
 'মানুষের',
 'বরাদ্দ',
 'কম',
 'করে',
 'দিয়ে',
 'নিজে

In [27]:
bnlp_word_tokens2

['চিত্ত',
 'যেথা',
 'ভয়শূন্য',
 'উচ্চ',
 'যেথা',
 'শির',
 'জ্ঞান',
 'যেথা',
 'মুক্ত',
 'যেথা',
 'গৃহের',
 'প্রাচীর',
 'আপন',
 'প্রাঙ্গণতলে',
 'দিবসশর্বরী',
 'বসুধারে',
 'রাখে',
 'নাই',
 'খণ্ড',
 'ক্ষুদ্র',
 'করি',
 'যেথা',
 'বাক্য',
 'হৃদয়ের',
 'উৎসমুখ',
 'হতে',
 'উচ্ছ্বসিয়া',
 'উঠে',
 'যেথা',
 'নির্বারিত',
 'স্রোতে',
 'দেশের',
 'হ্রদয়',
 'বুদ্ধির',
 'সূত্রে',
 'শত',
 'শাখা',
 'শত',
 'স্রোতে',
 'সেঁচিয়া',
 'উঠে',
 'যেথা',
 'কর্মযজ্ঞে',
 'বসন্তের',
 'হাওয়ায়',
 'অনুপ্রাণিত',
 'হয়ে',
 'অজানা',
 'পথে',
 'চলবারে',
 'জনে',
 'যেথা',
 'চিন্তা',
 'তোমার',
 'আচারপন্থাকে',
 'বুদ্ধির',
 'তীক্ষ্ণ',
 'তলওয়ারে',
 'প্রতি',
 'দিন',
 'আঘাত',
 'করে',
 'যেথা',
 'তুমি',
 'বিশ্ববিধাতা',
 'প্রবুদ্ধ',
 'উদার',
 'সৃষ্টি',
 'করে',
 'থাকো',
 'সর্বত্র',
 'সেই',
 'স্বর্গে',
 'উঠিতে',
 'যাক',
 'আমার',
 'দেশ']

In [28]:
lemmatized_result1 = light_pipeline.annotate(bnlp_word_tokens1)
lemmatized_result2 = light_pipeline.annotate(bnlp_word_tokens2)

In [29]:
lemmatized_result1

[{'document': ['জীবের'], 'token': ['জীবের'], 'lemma': ['জীব']},
 {'document': ['মধ্যে'], 'token': ['মধ্যে'], 'lemma': ['মধ্যে']},
 {'document': ['সবচেয়ে'], 'token': ['সবচেয়ে'], 'lemma': ['সবচেয়ে']},
 {'document': ['সম্পূর্ণতা'],
  'token': ['সম্পূর্ণতা'],
  'lemma': ['সম্পূর্ণতা']},
 {'document': ['মানুষের'], 'token': ['মানুষের'], 'lemma': ['মানুষ']},
 {'document': ['কিন্তু'], 'token': ['কিন্তু'], 'lemma': ['কিন্তু']},
 {'document': ['সবচেয়ে'], 'token': ['সবচেয়ে'], 'lemma': ['সবচেয়ে']},
 {'document': ['অসম্পূর্ণ'], 'token': ['অসম্পূর্ণ'], 'lemma': ['অসম্পূর্ণ']},
 {'document': ['হয়ে'], 'token': ['হয়ে'], 'lemma': ['হয়ে']},
 {'document': ['সে'], 'token': ['সে'], 'lemma': ['সে']},
 {'document': ['জন্মগ্রহণ'], 'token': ['জন্মগ্রহণ'], 'lemma': ['জন্মগ্রহণ']},
 {'document': ['করে'], 'token': ['করে'], 'lemma': ['করা']},
 {'document': ['বাঘ'], 'token': ['বাঘ'], 'lemma': ['বাঘ']},
 {'document': ['ভালুক'], 'token': ['ভালুক'], 'lemma': ['ভালুক']},
 {'document': ['তার'], 'token': ['তার'],

In [30]:
lemmatized_result2

[{'document': ['চিত্ত'], 'token': ['চিত্ত'], 'lemma': ['চিত্ত']},
 {'document': ['যেথা'], 'token': ['যেথা'], 'lemma': ['যেথা']},
 {'document': ['ভয়শূন্য'], 'token': ['ভয়শূন্য'], 'lemma': ['ভয়শূন্য']},
 {'document': ['উচ্চ'], 'token': ['উচ্চ'], 'lemma': ['উচ্চ']},
 {'document': ['যেথা'], 'token': ['যেথা'], 'lemma': ['যেথা']},
 {'document': ['শির'], 'token': ['শির'], 'lemma': ['শির']},
 {'document': ['জ্ঞান'], 'token': ['জ্ঞান'], 'lemma': ['জ্ঞান']},
 {'document': ['যেথা'], 'token': ['যেথা'], 'lemma': ['যেথা']},
 {'document': ['মুক্ত'], 'token': ['মুক্ত'], 'lemma': ['মুক্ত']},
 {'document': ['যেথা'], 'token': ['যেথা'], 'lemma': ['যেথা']},
 {'document': ['গৃহের'], 'token': ['গৃহের'], 'lemma': ['গৃহ']},
 {'document': ['প্রাচীর'], 'token': ['প্রাচীর'], 'lemma': ['প্রাচীর']},
 {'document': ['আপন'], 'token': ['আপন'], 'lemma': ['আপন']},
 {'document': ['প্রাঙ্গণতলে'],
  'token': ['প্রাঙ্গণতলে'],
  'lemma': ['প্রাঙ্গণতলে']},
 {'document': ['দিবসশর্বরী'],
  'token': ['দিবসশর্বরী'],
  'lemma': 

In [31]:
for dict in lemmatized_result1:
  print(dict["lemma"])

['জীব']
['মধ্যে']
['সবচেয়ে']
['সম্পূর্ণতা']
['মানুষ']
['কিন্তু']
['সবচেয়ে']
['অসম্পূর্ণ']
['হয়ে']
['সে']
['জন্মগ্রহণ']
['করা']
['বাঘ']
['ভালুক']
['তিনি']
['জীবনযাত্রা']
['পনেরো', '-']
['আনা']
['মূলধন']
['নিয়ে']
['আসা']
['প্রকৃত']
['মালখানা']
['থেকে']
['জীবরঙ্গভূমিতে']
['মানুষ']
['আসা']
['দেখা']
['দেয়']
['দুই']
['শূন্য']
['হাত']
['মুঠো']
['বাঁধা']
['মানুষ']
['আসবার']
['পূর্ববল']
['জীবসৃষ্টিযজ্ঞে']
['প্রকৃত']
['ভূরিব্যয়ের']
['পালা']
['শেষ']
['হয়ে']
['আসা']
['বিপুল']
['মাংস']
['কঠিন']
['বর্ম']
['প্রকাণ্ড']
['লেজ']
['নিয়ে']
['জল']
['স্থলে']
['পৃথুল']
['দেহের']
['যে']
['অমিতাচার']
['প্রবল']
['হয়ে']
['ওঠা']
['তা']
['ধরিত্রীকে']
['দেওয়া']
['ক্লান্ত']
['করা']
['প্রমাণ']
['হল']
['আতিশয্যের']
['পরাভব']
['অনিবার্য']
['পরীক্ষায়']
['এটাও']
['স্থির']
['হয়ে']
['গেল']
['যে']
['প্রশ্রয়ের']
['পরিমাণ']
['যত']
['বেশি']
['হয়']
['দুর্বলতার']
['বোঝাও']
['তত']
['দুর্বহ']
['হয়ে']
['ওঠা']
['নূতন']
['পর্ব']
['প্রকৃত']
['যথাসম্ভব']
['মানুষ']
['বরাদ্দ']
['কম']
['করা']
['দিয়ে']
['নিজ']
['রওয়া']
['নেপ

In [33]:
for dict in lemmatized_result2:
  print(dict["lemma"])

['চিত্ত']
['যেথা']
['ভয়শূন্য']
['উচ্চ']
['যেথা']
['শির']
['জ্ঞান']
['যেথা']
['মুক্ত']
['যেথা']
['গৃহ']
['প্রাচীর']
['আপন']
['প্রাঙ্গণতলে']
['দিবসশর্বরী']
['বসুধারে']
['রাখা']
['না']
['খণ্ড']
['ক্ষুদ্র']
['করা']
['যেথা']
['বাক্য']
['হৃদয়ের']
['উৎসমুখ']
['হতে']
['উচ্ছ্বসিয়া']
['ওঠা']
['যেথা']
['নির্বারিত']
['স্রোত']
['দেশ']
['হ্রদয়']
['বুদ্ধি']
['সূত্র']
['শত']
['শাখা']
['শত']
['স্রোত']
['সেঁচিয়া']
['ওঠা']
['যেথা']
['কর্মযজ্ঞে']
['বসন্ত']
['হাওয়ায়']
['অনুপ্রাণিত']
['হয়ে']
['অজানা']
['পথ']
['চলবারে']
['জনে']
['যেথা']
['চিন্তা']
['তুমি']
['আচারপন্থাকে']
['বুদ্ধি']
['তীক্ষ্ণ']
['তলওয়ারে']
['প্রতি']
['দেওয়া']
['আঘাত']
['করা']
['যেথা']
['তুমি']
['বিশ্ববিধাতা']
['প্রবুদ্ধ']
['উদার']
['সৃষ্টি']
['করা']
['থাকো']
['সর্বত্র']
['সেই']
['স্বর্গ']
['ওঠা']
['যাওয়া']
['আমি']
['দেশ']
